In [17]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from IPython.core.debugger import set_trace
import numpy as np

In [18]:
# Dataset for pairs
class BoxDataset(Dataset):
  """Dataset wrapping images and target labels for Kaggle - Planet Amazon from Space competition.

  Arguments:
      A CSV file path
  """

  def __init__(self, csv_path):
    data = np.loadtxt(csv_path)
    self.len = len(data)
    self.X_train = torch.from_numpy(data[:,:2].astype(np.long))
    self.y_train = torch.from_numpy(data[:,2].astype(np.float32))

  def __getitem__(self, index):
    return self.X_train[index], self.y_train[index]

  def __len__(self):
    return self.len

# Model = a tensor of boxes
class Boxes(nn.Module):
  def __init__(self, num_boxes, dim):
    super(Boxes, self).__init__()
    box_mins = torch.rand(num_boxes, dim)
    box_maxs = box_mins + torch.rand(num_boxes, dim) * (1 - box_mins)
    boxes = torch.stack([box_mins, box_maxs], dim=1)
    self.boxes = nn.Parameter(boxes)
    
  def forward(self, X):
    """Returns box embeddings for ids"""
    #set_trace()
    x = self.boxes[X]
    o = cond_probs(x[:,0,:,:], x[:,1,:,:])
    return o
    

In [19]:
MIN_IND, MAX_IND = 0, 1

def volumes(boxes):
  r = (boxes[:,MAX_IND,:] - boxes[:, MIN_IND,:]).clamp(0).prod(1)
  return r

def intersections(boxes1, boxes2):
  #set_trace()
  intersections_min = torch.max(boxes1[:, :, MIN_IND], boxes2[:, :, MIN_IND])
  intersections_max = torch.min(boxes1[:, :, MAX_IND], boxes2[:, :, MAX_IND])
  apap = torch.stack([intersections_min, intersections_max], 1)
  return apap

def cond_probs(boxes1, boxes2):
  return volumes(intersections(boxes1, boxes2))/volumes(boxes2)

In [20]:
train_ds = BoxDataset("data/sample/train.txt")
train_dl = DataLoader(train_ds, batch_size=18, shuffle=True, num_workers=4)

model = Boxes(6,4)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [22]:
N_EPOCHS = 100

for epoch in range(N_EPOCHS):
    
    # Train
    model.train()  # IMPORTANT
    
    running_loss, correct = 0.0, 0
    for X, y in train_dl:
      #set_trace()
      #X, y = X.to(device), y.to(device)

      optimizer.zero_grad()
      with torch.set_grad_enabled(True):
        y_ = model(X)
        loss = criterion(y_, y)

      loss.backward()
      
      for param in model.parameters():
          print(param.grad.data.sum())
      print(model.boxes[0])
      optimizer.step()
      
      # Statistics
      print("    batch loss: "+str(loss.item()))
      #_, y_label_ = torch.max(y_, 1)
      #correct += (y_label_ == y).sum().item()
      running_loss += loss.item() * X.shape[0]

    print("  Train Loss: "+str(running_loss / len(train_dl.dataset)))
    print("  Train Acc:  "+str(correct / len(train_dl.dataset)))
    

tensor(0.)
tensor([[0.7772, 0.6410, 0.3975, 0.5548],
        [0.8129, 0.7153, 0.5910, 0.6832]], grad_fn=<SelectBackward>)
    batch loss: 11443.5839844
  Train Loss: 11443.5839844
  Train Acc:  0
tensor(0.)
tensor([[0.7772, 0.6410, 0.3975, 0.5548],
        [0.8129, 0.7153, 0.5910, 0.6832]], grad_fn=<SelectBackward>)
    batch loss: 10124.4345703
  Train Loss: 10124.4345703
  Train Acc:  0
tensor(0.)
tensor([[0.7772, 0.6410, 0.3975, 0.5548],
        [0.8129, 0.7153, 0.5910, 0.6832]], grad_fn=<SelectBackward>)
    batch loss: 9092.77929688
  Train Loss: 9092.77929688
  Train Acc:  0
tensor(0.1250)
tensor([[0.7772, 0.6410, 0.3975, 0.5548],
        [0.8129, 0.7153, 0.5910, 0.6832]], grad_fn=<SelectBackward>)
    batch loss: 8269.88476562
  Train Loss: 8269.88476562
  Train Acc:  0
tensor(0.)
tensor([[0.7772, 0.6410, 0.3975, 0.5548],
        [0.8129, 0.7153, 0.5910, 0.6832]], grad_fn=<SelectBackward>)
    batch loss: 7602.48583984
  Train Loss: 7602.48583984
  Train Acc:  0
tensor(-0.2500)


tensor(0.0625)
tensor([[0.7772, 0.6410, 0.3975, 0.5548],
        [0.8129, 0.7153, 0.5910, 0.6832]], grad_fn=<SelectBackward>)
    batch loss: 3572.01953125
  Train Loss: 3572.01953125
  Train Acc:  0
tensor(0.)
tensor([[0.7772, 0.6410, 0.3975, 0.5548],
        [0.8129, 0.7153, 0.5910, 0.6832]], grad_fn=<SelectBackward>)
    batch loss: 3565.43115234
  Train Loss: 3565.43115234
  Train Acc:  0
tensor(0.)
tensor([[0.7772, 0.6410, 0.3975, 0.5548],
        [0.8129, 0.7153, 0.5910, 0.6832]], grad_fn=<SelectBackward>)
    batch loss: 3559.44213867
  Train Loss: 3559.44213867
  Train Acc:  0
tensor(-0.0625)
tensor([[0.7772, 0.6410, 0.3975, 0.5548],
        [0.8129, 0.7153, 0.5910, 0.6832]], grad_fn=<SelectBackward>)
    batch loss: 3554.04394531
  Train Loss: 3554.04394531
  Train Acc:  0
tensor(0.)
tensor([[0.7772, 0.6410, 0.3975, 0.5548],
        [0.8129, 0.7153, 0.5910, 0.6832]], grad_fn=<SelectBackward>)
    batch loss: 3549.15209961
  Train Loss: 3549.15209961
  Train Acc:  0
tensor(0.)


tensor(0.)
tensor([[0.7772, 0.6410, 0.3975, 0.5548],
        [0.8129, 0.7153, 0.5910, 0.6832]], grad_fn=<SelectBackward>)
    batch loss: 3499.04150391
  Train Loss: 3499.04150391
  Train Acc:  0
tensor(-0.0625)
tensor([[0.7772, 0.6410, 0.3975, 0.5548],
        [0.8129, 0.7153, 0.5910, 0.6832]], grad_fn=<SelectBackward>)
    batch loss: 3498.85180664
  Train Loss: 3498.85180664
  Train Acc:  0
tensor(0.)
tensor([[0.7772, 0.6410, 0.3975, 0.5548],
        [0.8129, 0.7153, 0.5910, 0.6832]], grad_fn=<SelectBackward>)
    batch loss: 3498.66357422
  Train Loss: 3498.66357422
  Train Acc:  0
tensor(0.)
tensor([[0.7772, 0.6410, 0.3975, 0.5548],
        [0.8129, 0.7153, 0.5910, 0.6832]], grad_fn=<SelectBackward>)
    batch loss: 3498.47485352
  Train Loss: 3498.47485352
  Train Acc:  0
tensor(0.)
tensor([[0.7772, 0.6410, 0.3975, 0.5548],
        [0.8129, 0.7153, 0.5910, 0.6832]], grad_fn=<SelectBackward>)
    batch loss: 3498.28759766
  Train Loss: 3498.28759766
  Train Acc:  0
tensor(-0.0625)